# Recommender Systems  Matrix Factorization 
## Introduction
In this task, we will be working on the MovieLens 1M dataset which can be fetched from moivelens . This set contains about 1.000.000 ratings given to about 4.000 movies by about 6.000 users. Our task adopted Matrix Factorization Algorithm in Python and estimated the accuracy with the Root Mean Squared Error, RMSE, and the Mean Absolute Error, MAE.

To make sure that the results are reliable, 5-fold cross- validation is used.

## Gradient Descend
This was described in the paper gravity-Tikk.pdf (the beginning of section 3.1)

In [45]:
import numpy as np
import pandas as pd
import math
import random
from sklearn.model_selection import KFold

from math import sqrt

#Import libs to plot the training process
import matplotlib
import matplotlib.pyplot as plt

#Set the random seed to make the experiment repeatable
np.random.seed(2019)
random.seed(2019)

## Data
The CSV file ratings.csv contains 100836 ratings created by 610 users about 9742 movies.

In [180]:
#Define the file path
file_path = '../data/ratings.csv'#"./ml-latest-small/ratings.csv" 
df_ratings = pd.read_csv(file_path, header=0, names=['userId', 'movieId', 'rating'], index_col=False)

In [181]:
# Renumbering user and movie IDs
uniqueMovieIds = df_ratings.movieId.unique() #Obtain the unique movie Ids
uniqueUserIds = df_ratings.userId.unique() #Obtain the unique user Ids

#Create New movieIds according to unique movie Ids
newMovieIds = [np.where(uniqueMovieIds==x)[0][0] for x in df_ratings['movieId'].tolist()]
newMovieIds = np.array(newMovieIds)

#Create New userIds according to unique user Ids
newUserIds = [np.where(uniqueUserIds==x)[0][0] for x in df_ratings['userId'].tolist()]
newUserIds = np.array(newUserIds)

In [183]:
#Add new movieIds and new userIds to the dataframe
df_ratings['newMovieId'] = newMovieIds
df_ratings['newUserId'] = newUserIds

#Reconstruct the dataframe
df_ratings = df_ratings[["newUserId","newMovieId","rating"]]

In [184]:
#Construct The Rating Matrix acorting to the new ids
num_users = df_ratings['newUserId'].max()+1  #There are 610 users
num_movies = df_ratings['newMovieId'].max()+1 #The Max Value of movieId

ratings = np.zeros((num_users , num_movies))

print(ratings.shape)

for i in range(df_ratings.shape[0]):
    ratings[df_ratings.loc[i, 'newUserId']][df_ratings.loc[i,'newMovieId']] = df_ratings.loc[i, 'rating']

(610, 9724)


In [202]:
class MatrixFactorization():
    def __init__( self, R=ratings, df=df_ratings, K=20, lr=0.001, rp=0.01, itr= 3, nfolds=5): 
        #change itr and set thresholds
        """
        Arguments/Hyperparamters
        -R: user-movie: rating matrix
        -K: number of latent dimensions of user & movie vectors
        -lr: learning rate
        -rp: regularization parametre
        -itr: number of iterations
        """     
        self.R=R
        self.df = df
        self.num_users, self.num_movies = R.shape
        self.K = K
        self.lr = lr
        self.rp = rp
        self.itr = itr
        self.nfolds = nfolds
              
    def train_n_times(self):
        # set up the empty result list
        err=[]
        kf = KFold(n_splits= self.nfolds, shuffle=True)
        n_t = 1
        for train_index, test_index in kf.split(self.df):
            print('Training No.', n_t)
            n_t += 1
            training_set, test_set = self.df.iloc[train_index], self.df.iloc[test_index]
            training_process = self.train(training_set, test_set)
            err.append(training_process )
        return (err)
            
    def gradient_descent(self, training):
        #Update the parametres of user&movie matrix in one iteration
        for u, m, r in training:
            if r:
                error = r - self.predict_um(u,m)
                self.U[u] += self.lr*(error*self.M[m]-self.rp*self.U[u])
                self.M[m] += self.lr*(error*self.U[u] - self.rp*self.M[m])

    def train(self, training_set, test_set):
        #Initialize user & movie matrices 
        self.U = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.M = np.random.normal(scale=1./self.K, size=(self.num_movies, self.K))
        
        #Perform stochastic gradient descent 
        training_process = []
        training = training_set.as_matrix().astype(int)
        test = test_set.as_matrix().astype(int)
        for i in range(self.itr):
            self.gradient_descent(training)
            mse_training = self.calculate_mse(training)
            mse_test = self.calculate_mse(test)
            training_process.append((i, mse_training, mse_test))
        
            print("Iteration: %d; mse_traing: %.4f; mse_test: %.4f"%(i+1, mse_training, mse_test))
            
        iterations = [i for i, mse_training, mse_test in training_process]
        mse_training = [mse_training for i, mse_training, mse_test in training_process]
        mse_test = [mse_test for i, mse_training, mse_test in training_process]
        plt.figure(figsize=(16, 4))
        plt.title("Training Error And Validation Error")
        plt.plot(iterations, mse_training, color='blue', label = 'training error')
        plt.plot(iterations, mse_test, color='orange', label='validation error')
        plt.xticks(iterations, iterations)
        plt.xlabel("Iterations")
        plt.ylabel("Mean Square Error")
        plt.legend()
        plt.grid(axis="y")
        #plt.show()
        return training_process
            
            
    def round_of_rating(self, R):
        matrixRound = np.vectorize(round) #Vectorize the round function
        return matrixRound(R*2)/2 # Round off the rating to nearest 0.5
    
    
    def predictAll(self):
        #Predict All the Ratings, return a full rating matrix
        predictions = self.U.dot(self.M.T)
        return self.round_of_rating(predictions)       
    
    def predict_um(self, u, m):
        #Predict the rating of user u to movie m
        prediction = self.U[u,:].dot(self.M[m].T)
        return prediction
    
    def calculate_mse(self, d):
        #Calculate mse for Training/Testing set
        #train_pred['diff'] = (d.rating - self.predictALL))**2
        #err_train.append(np.sqrt(train_pred['diff'].mean()))
        num_of_d = len(d)
        error = 0
        
        for i, j, r in d:
            prediction = self.predict_um(i,j)
            error += (prediction - r )**2
            
        error = error/num_of_d
        return error

In [239]:
mf = MatrixFactorization()
err = mf.train_n_times()

Training No. 1
Iteration: 1; mse_traing: 12.4048; mse_test: 12.4435


KeyboardInterrupt: 

In [201]:
err

[[(0, 12.400866423883944, 12.458552927438825),
  (1, 12.397691333876306, 12.457915461740132),
  (2, 12.393209249923883, 12.456109685562543)],
 [(0, 12.414368575944138, 12.403381906529853),
  (1, 12.411020019875348, 12.402630602271573),
  (2, 12.406137326369773, 12.40046142487515)],
 [(0, 12.401611440352163, 12.455829307251014),
  (1, 12.398713945853055, 12.455397420656423),
  (2, 12.394861416286737, 12.454062511976229)],
 [(0, 12.410941040542369, 12.41739103746136),
  (1, 12.407636063735653, 12.416662712916903),
  (2, 12.402890045202161, 12.41464182692429)],
 [(0, 12.431613185070495, 12.337056109916707),
  (1, 12.428642235921593, 12.336615012113691),
  (2, 12.424555316364138, 12.335177349952641)]]

In [233]:
result = np.mean(err, axis = 0)[2]

In [238]:
#print the final conclusion:
print("For the MF with GD ")
print("Mean error on TRAIN: " + str(result[1]))
print("Mean error on  TEST: " + str(result[2]))

For the MF with GD 
Mean error on TRAIN: 12.40499678832716
Mean error on  TEST: 12.412558218023136


## MF with ALS

In [ ]:
r_mat = df_ratings.pivot(index='userId', columns = 'movieId', values = 'rating')

U[:,0] = np.mean(r_mat, axis = 1)
M[:,0] = np.mean(r_mat, axis = 0)

UI = U[:,X[:,j].nonzero()[0]]

for u in range(num_users):
    vec = X[u,:].nonzero()[0]
    MI = M[vec,:]
    n = max(len(vec),1)
    A = np.dot(MI.T,MI) + lmda*n*np.eye(num_features)
    V = np.dot(MI.T,X[u,X[u,:].nonzero()[0]])
    U[u,:] = np.linalg.solve(A,V)
        
for m in range(num_movies):
    vec = X[:,m].nonzero()[0]
    UI = U[vec,:]
    n = max(len(vec),1)
    A = np.dot(UI.T,UI) + lmda*n*np.eye(num_features)
    V = np.dot(UI.T,X[X[:,m].nonzero()[0],j])
    M[m,:] = np.linalg.solve(A,V)
        
pred = np.dot(U.T,M)
    